In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt

%matplotlib inline


In [5]:
df = pd.read_csv("/label.txt",sep=",,,",header=None ,names=['question','type'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [6]:
df.head()

,question,type
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [7]:
df.shape

(1483, 2)

In [8]:
df['type']=df['type'].str.strip()

In [9]:
df['type'].unique()

array(['unknown', 'what', 'when', 'who', 'affirmation'], dtype=object)

In [10]:
df['question'].values

array(['how did serfdom develop in and then leave russia ? ',
       'what films featured the character popeye doyle ? ',
       "how can i find a list of celebrities ' real names ? ", ...,
       'does this hose have one ? ', 'can i get it in india ? ',
       'would this work on a 2008 ford edge with a naked roof ? '],
      dtype=object)

In [12]:
import re
df['question'] = df['question'].apply(lambda x: x.lower())
df['question'] = df['question'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [13]:
VALIDATION_SPLIT=0.20

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [15]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=30

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

In [17]:
data=df.copy()

In [18]:
print(data['type'].value_counts())

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ')
tokenizer.fit_on_texts(data['question'].values)
X = tokenizer.texts_to_sequences(data['question'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

what           609
who            402
unknown        272
affirmation    104
when            96
Name: type, dtype: int64


In [59]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Y = data['type']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)


# split the data into a training set and a validation set
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X.shape[0])

x_train = X[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Found 3685 unique tokens.
Shape of data tensor: (1483, 30)
Shape of label tensor: (1483, 5)


In [39]:
!wget http://nlp.stanford.edu/data/glove.6B.zip 

--2020-08-22 10:58:23--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-22 10:58:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-22 10:58:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [40]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [41]:
%ls

glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
glove.6B.200d.txt  glove.6B.50d.txt   sample_data/


In [42]:
embeddings_index = {}
f = open('glove.6B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [43]:
EMBEDDING_DIM=300

In [44]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [45]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [48]:
embed_dim = 300
lstm_out = 196
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(lstm_out, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 300)           1105800   
_________________________________________________________________
lstm (LSTM)                  (None, 196)               389648    
_________________________________________________________________
dense (Dense)                (None, 5)                 985       
Total params: 1,496,433
Trainable params: 390,633
Non-trainable params: 1,105,800
_________________________________________________________________
None


In [49]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_val, y_val))

Epoch 1/20
10/10 [==============================] - 4s 420ms/step - loss: 1.3567 - accuracy: 0.4431 - val_loss: 1.1997 - val_accuracy: 0.5709
Epoch 2/20
10/10 [==============================] - 4s 382ms/step - loss: 1.0081 - accuracy: 0.6495 - val_loss: 0.7930 - val_accuracy: 0.7027
Epoch 3/20
10/10 [==============================] - 4s 381ms/step - loss: 0.6590 - accuracy: 0.7641 - val_loss: 0.5596 - val_accuracy: 0.8243
Epoch 4/20
10/10 [==============================] - 4s 386ms/step - loss: 0.4556 - accuracy: 0.8627 - val_loss: 0.3573 - val_accuracy: 0.8986
Epoch 5/20
10/10 [==============================] - 4s 372ms/step - loss: 0.3011 - accuracy: 0.9174 - val_loss: 0.2643 - val_accuracy: 0.9324
Epoch 6/20
10/10 [==============================] - 4s 366ms/step - loss: 0.2138 - accuracy: 0.9427 - val_loss: 0.1843 - val_accuracy: 0.9493
Epoch 7/20
10/10 [==============================] - 4s 365ms/step - loss: 0.1543 - accuracy: 0.9537 - val_loss: 0.1554 - val_accuracy: 0.9628
Epoch 

In [79]:

example = tokenizer.texts_to_sequences(["how did you come?"])
example = pad_sequences(example, maxlen=MAX_SEQUENCE_LENGTH)

In [80]:
le.inverse_transform([np.argmax(model.predict(example))])


array(['unknown'], dtype=object)